In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from Utils import Utils
from Constants import Constants
from Propensity_socre_network import Propensity_socre_network
from Utils import Utils
from PS_Manager import PS_Manager
from PS_Treated_Generator import PS_Treated_Generator

from GAN import Generator, Discriminator
from sklearn.neighbors import NearestNeighbors

from GAN_Manager import GAN_Manager
from Utils import Utils

from matplotlib import pyplot
from torch.autograd.variable import Variable
from collections import OrderedDict
from dataloader import DataLoader

In [2]:
def __get_ps_model(ps_model_type, iter_id,
                   input_nodes, device,
                   np_covariates_X_train, 
                   np_covariates_X_test, 
                   np_covariates_Y_train,
                   np_covariates_Y_test, dL):
    ps_train_set = dL.convert_to_tensor(np_covariates_X_train, np_covariates_Y_train)
    ps_test_set = dL.convert_to_tensor(np_covariates_X_test,
                                            np_covariates_Y_test)
    ps_manager = PS_Manager()
    if ps_model_type == Constants.PS_MODEL_NN:
        return ps_manager.get_propensity_scores(ps_train_set,
                                                    ps_test_set, iter_id,
                                                    input_nodes, device)

In [11]:
iter_id = 1

In [3]:
input_nodes = 30
device = Utils.get_device()
ps_model_type=Constants.PS_MODEL_NN
csv_path = "Dataset/Twin_data.csv"
split_size = 0.8
dL = DataLoader()

Constants.PROP_SCORE_NN_EPOCHS

1000

In [4]:
np_covariates_X_train, np_covariates_X_test, np_covariates_Y_train, \
        np_covariates_Y_test \
            = dL.preprocess_data_from_csv(csv_path, split_size)
            # get propensity score for classifier training and testing


(11400, 30)
(11400, 2)
(11400, 1)
(11400, 1)
(11400, 33)


In [5]:
ps_score_list_train, ps_score_list_test, ps_model = __get_ps_model(ps_model_type, 
                                                                   1, input_nodes,
                                                                device, 
                                                                np_covariates_X_train, 
                                                                np_covariates_X_test, 
                                                                np_covariates_Y_train,
                                                                np_covariates_Y_test, dL)


############### Propensity Score neural net Training ###############
.. PS Training started ..
Epoch: 100, loss: 192.15677219629288, correct: 5052/9120, accuracy: 0.5539473684210526
Epoch: 200, loss: 186.045654296875, correct: 5291/9120, accuracy: 0.5801535087719298
Epoch: 300, loss: 181.5715565085411, correct: 5406/9120, accuracy: 0.5927631578947369
Epoch: 400, loss: 179.31907391548157, correct: 5483/9120, accuracy: 0.6012061403508772
Epoch: 500, loss: 177.23652535676956, correct: 5561/9120, accuracy: 0.6097587719298245
Epoch: 600, loss: 174.92337548732758, correct: 5606/9120, accuracy: 0.6146929824561403
Epoch: 700, loss: 175.74198299646378, correct: 5608/9120, accuracy: 0.6149122807017544
Epoch: 800, loss: 175.03584891557693, correct: 5669/9120, accuracy: 0.6216008771929824
Epoch: 900, loss: 172.75191229581833, correct: 5635/9120, accuracy: 0.6178728070175439
Epoch: 1000, loss: 172.39621379971504, correct: 5615/9120, accuracy: 0.6156798245614035
Training Completed..


In [9]:
print("Train size: ")
data_loader_dict_train = dL.prepare_tensor_for_DCN(np_covariates_X_train,
                                                                    np_covariates_Y_train,
                                                                    ps_score_list_train,
                                                                    False)
print("Test size: ")
data_loader_dict_test = dL.prepare_tensor_for_DCN(np_covariates_X_test,
                                                                   np_covariates_Y_test,
                                                                   ps_score_list_test,
                                                                   False)
tensor_treated_train_original = \
                Utils.create_tensors_from_tuple(data_loader_dict_train["treated_data"])
tensor_control_train_original = \
                Utils.create_tensors_from_tuple(data_loader_dict_test["control_data"])

            # Execute PM GAN
ps_t = PS_Treated_Generator(data_loader_dict_train, ps_model)

Train size: 
Big X: (9120, 35)
 Treated Statistics ==>
(4578, 30)
 Control Statistics ==>
(4542, 30)
Test size: 
Big X: (2280, 35)
 Treated Statistics ==>
(1183, 30)
 Control Statistics ==>
(1097, 30)


In [12]:
balanced_dataset_dict = ps_t.simulate_treated_semi_supervised(input_nodes, iter_id, device)
            

-> Matched Control: (4578, 30)
-> UnMatched Control: (2645, 30)
-> GAN training started


KeyboardInterrupt: 